In [119]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Creating Audio File

Checkout https://beta.elevenlabs.io/history for full log of audio creation

In [211]:
eleven_labs_api = "aa42307ce6bfbefcfd2abd4d8634127b"
file_name = "./obama_intro"
mp3_file_name = f"../AAAI22-one-shot-talking-face/samples/audios/{file_name}.mp3"
wav_file_name = f"../AAAI22-one-shot-talking-face/samples/audios/{file_name}.wav"

generate_new_audio = False
name_of_voice = "T"
text = "Good evening, everyone. Donald Trump here bringing you the latest news from the upside down world. "

In [212]:
# Get voice ID
import requests

headers = {'xi-api-key': eleven_labs_api}
r1 = requests.get('https://api.elevenlabs.io/v1/voices', headers=headers)
resp = r1.json()
rid = ""
for voice in resp["voices"]:
    if voice["name"] == name_of_voice:
        print(voice)
        rid = voice["voice_id"]
        break;
rid

{'voice_id': 'hAvYeODVB0NpGQX2vvYo', 'name': 'T', 'samples': [{'sample_id': 'MUq3YryANBS8VuL1vxv4', 'file_name': 'trump_strong.mp3', 'mime_type': 'audio/mpeg', 'size_bytes': 826788, 'hash': '031b0827047c91bd77243c3a9d7813c3'}, {'sample_id': 'Ug3mHySfs2ubWOjd69mq', 'file_name': 'Trump_WEF_2018 (mp3cut.net).mp3', 'mime_type': 'audio/mpeg', 'size_bytes': 4900013, 'hash': 'e5154651e4aa8d0aba74fd33cd5ea280'}, {'sample_id': 'jfBQ1LOIbChk30UoFwa1', 'file_name': 'trump.mp3', 'mime_type': 'audio/mpeg', 'size_bytes': 1856140, 'hash': 'ce297ae4d8ce548be4528a4a56afa98c'}], 'category': 'cloned', 'fine_tuning': {'is_allowed_to_fine_tune': False, 'fine_tuning_requested': False, 'finetuning_state': 'not_started', 'verification_attempts': None, 'verification_attempts_count': 0}, 'labels': {}, 'preview_url': 'https://storage.googleapis.com/eleven-public-prod/8t6uB5rLIfPz6K2jM5kLIdrKZSl1/voices/hAvYeODVB0NpGQX2vvYo/9cb2beb3-b2aa-4bc0-aa46-62f3b3559a96.mp3', 'available_for_tiers': [], 'settings': None}


'hAvYeODVB0NpGQX2vvYo'

In [173]:
# Get audio file, minimize runs cost money
import json

if generate_new_audio:
    url = "https://api.elevenlabs.io/v1/text-to-speech/{voice_id}".format(voice_id=rid)
    print(url)
    headers = {'xi-api-key': eleven_labs_api}
    payload = {
      "text": text,
      "voice_settings": {
        "stability": 0,
        "similarity_boost": 0.5
      }
    }

    r2 = requests.post(url, data=json.dumps(payload), headers=headers)
    print(r2)

https://api.elevenlabs.io/v1/text-to-speech/hAvYeODVB0NpGQX2vvYo
<Response [200]>


In [207]:
# Save and play audio file
import os

if generate_new_audio:
    with open(mp3_file_name, 'wb') as f:
        f.write(r2.content)

    os.system("mpg123 " + mp3_file_name)

High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.31.2; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Directory: ../AAAI22-one-shot-talking-face/samples/audios/./
Playing MPEG stream 1 of 1: trump_intro.mp3 ...

MPEG 1.0 L III cbr64 44100 mono

[0:05] Decoding of trump_intro.mp3 finished.


In [208]:
# convert mp3 to wav                                                            
from pydub import AudioSegment
from pydub.playback import play

if generate_new_audio:
    sound = AudioSegment.from_mp3(mp3_file_name)
    sound.export(wav_file_name, format="wav")

    # test sound
    song = AudioSegment.from_wav(wav_file_name)
    play(song)

Input #0, wav, from '/var/folders/g8/b65pczf97pb17p1_zrldsm500000gn/T/tmp0ov9bh8k.wav':
  Duration: 00:00:05.93, bitrate: 705 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 1 channels, s16, 705 kb/s


## Generate phonemes
- Uses https://github.com/m-bain/whisperX

In [213]:
# no extension, must be wav
import os
cwd = os.getcwd() #pwd
print(cwd)

audio_file_name = wav_file_name
base_audio_file_name = os.path.splitext(os.path.basename(audio_file_name))[0]
print("File name:", audio_file_name)
print("Base name:", base_audio_file_name)

/Users/jfuentes/Projects/infinitenews/ForcedAlignment
File name: ../AAAI22-one-shot-talking-face/samples/audios/./obama_intro.wav
Base name: obama_intro


In [214]:
import gc
import torch
import whisperx

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transcribe with original whisper
model = whisperx.load_model("medium.en", device)
# model = whisperx.load_model("base.en", device)

In [215]:
result = model.transcribe(audio_file_name)
print(result["segments"]) # before alignment

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/whisperx/transcribe.py:83: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.0, 'text': ' Good evening, everyone. Obama here bringing you the latest news from the upside-down world.', 'tokens': [50363, 4599, 6180, 11, 2506, 13, 2486, 994, 6079, 345, 262, 3452, 1705, 422, 262, 17196, 12, 2902, 995, 13, 50713], 'temperature': 0.0, 'avg_logprob': -0.27768791805614124, 'compression_ratio': 1.0833333333333333, 'no_speech_prob': 0.03393666073679924}]


In [216]:
# load alignment model and metadata
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)

# align whisper output
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file_name, device)

# print(result_aligned["segments"]) # after alignment
result_aligned["word_segments"]

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/whisperx/alignment.py:302: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  char_segments_arr = per_seg_grp.apply(lambda x: x.reset_index(drop = True)).reset_index()


[{'text': 'Good', 'start': 1.0429799426934099, 'end': 1.2034383954154728},
 {'text': 'evening,', 'start': 1.2836676217765044, 'end': 1.5644699140401148},
 {'text': 'everyone.', 'start': 1.6446991404011462, 'end': 2.025787965616046},
 {'text': 'Obama', 'start': 2.928366762177651, 'end': 3.3094555873925504},
 {'text': 'here', 'start': 3.349570200573066, 'end': 3.530085959885387},
 {'text': 'bringing', 'start': 3.5902578796561606, 'end': 3.891117478510029},
 {'text': 'you', 'start': 3.9312320916905446, 'end': 4.07163323782235},
 {'text': 'the', 'start': 4.111747851002866, 'end': 4.191977077363897},
 {'text': 'latest', 'start': 4.232091690544413, 'end': 4.553008595988539},
 {'text': 'news', 'start': 4.633237822349571, 'end': 4.914040114613181},
 {'text': 'from', 'start': 4.974212034383955, 'end': 5.134670487106018},
 {'text': 'the', 'start': 5.154727793696275, 'end': 5.255014326647565},
 {'text': 'upside-down', 'start': 5.335243553008596, 'end': 6.117478510028654},
 {'text': 'world.', 'sta

In [217]:
test_words = ["that's", "Help! ", "multiverse", "cat", "permit", "surprise", ",whales"]

In [218]:
import nltk

try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

In [219]:
import re
from functools import lru_cache
from itertools import product as iterprod

def cleanWord(word):
    regex = r"[^\w\']"
    return re.sub(regex, "", word).lower()

def arpabetWithoutNumbers(word):
    list_of_phones = []
    for phones in arpabet[word]:
        new_phones = ["".join(filter(lambda x: x.isalpha(), phone)) for phone in phones]
        list_of_phones.append(new_phones)
    return list_of_phones

def wordbreak(s):
    s = cleanWord(s)
    if s in arpabet:
        return arpabetWithoutNumbers(s)
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabetWithoutNumbers(pre), wordbreak(suf))]
    return None

# Example words: 
for w in test_words:
    print(wordbreak(w))

[['DH', 'AE', 'T', 'S']]
[['HH', 'EH', 'L', 'P']]
[['M', 'AH', 'L', 'T', 'IY', 'V', 'ER', 'S']]
[['K', 'AE', 'T']]
[['P', 'ER', 'M', 'IH', 'T'], ['P', 'ER', 'M', 'IH', 'T']]
[['S', 'ER', 'P', 'R', 'AY', 'Z'], ['S', 'AH', 'P', 'R', 'AY', 'Z']]
[['W', 'EY', 'L', 'Z'], ['HH', 'W', 'EY', 'L', 'Z']]


In [220]:
def letterToPhone(word, phones):
    # print("LTP", word, phones)
    if word == "":
        return []

    for i in reversed(range(1, len(word))):
        for j in range(0, len(word) - i + 1):
            subword = word[j:i+j]
            cur_possible_phones = wordbreak(subword)
            #For when has contractions
            if not cur_possible_phones:
#                 print("Skipping", i, j, subword, cur_possible_phones)
                continue
            else:
                cur_possible_phones = cur_possible_phones + [[subword.upper()]]
#                 print(i, j, subword, cur_possible_phones)

            for cur_phones in cur_possible_phones:
                extra_length = len(phones) - len(cur_phones)
#                 print("EL", extra_length)
                if extra_length > 0:
                    for a in range(0, extra_length + 1):
                        subphones_end = len(phones) - (extra_length - a)
                        subphones = phones[a:subphones_end]
                        if len(subphones) != len(cur_phones):
                            print("Lengths not equal....")
                        #print(subphones, cur_phones)
                        if subphones == cur_phones:
                            #print("MATCH", subword, subphones)
                            first = letterToPhone(word[0:j], phones[0:a])
                            cur = letterToPhone(subword, subphones)
                            last = letterToPhone(word[i+j:], phones[subphones_end:])
                            # print("LTP Complete", first, cur, last)
                            return first + cur + last
                    
    return [[word.upper(), phones]]

for segment in result_aligned["word_segments"]:
    word = segment["text"]
# for word in test_words:
#     print(word)
#     print(letterToPhone(word, wordbreak(word)[0]))
#     print([word, wordbreak(word)[0]], "\n")

In [221]:
chunked_words = []
for segment in result_aligned["word_segments"]:
    word = segment["text"]
    chunked_words.append(letterToPhone(word, wordbreak(word)[0]))

In [222]:
chunked_words[0]

[['G', ['G']], ['OO', ['UH']], ['D', ['D']]]

# Parsing out the transcriptions

In [223]:
ts = [o['text'] for o in result_aligned["word_segments"]]

In [224]:
ts = '|'.join(ts).upper()
ts = ts.replace(',', '')
ts = ts.replace('.', '')

In [225]:
ts

'GOOD|EVENING|EVERYONE|OBAMA|HERE|BRINGING|YOU|THE|LATEST|NEWS|FROM|THE|UPSIDE-DOWN|WORLD'

# FORCED ALIGNMENT FROM WAV2VEC2

In [226]:
import os
from dataclasses import dataclass

import torch
import torchaudio

import IPython
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["figure.figsize"] = [16.0, 4.8]

In [227]:
# model we are loading
MODEL_NAME = 'WAV2VEC2_ASR_BASE_LV60K_960H'
MODEL_NAME = 'WAV2VEC2_ASR_BASE_960H'
# sampling rate the model expects
# NOTE: most wav2vec models assume 16k sampling rate
MODEL_SR = 16_000


@dataclass
class Point:
    token_index: int
    time_index: int
    score: float


# Merge the labels
@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float

    def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start:5f}, {self.end:5f})"

    @property
    def length(self):
        return self.end - self.start



def load_audio(wav_file, model_sr=MODEL_SR):
    '''Loads `wav_file` using torchaudio.
    
    Resamples the audio to `MODEL_SR` if needed.
    '''
    wav, sr = torchaudio.load(wav_file)
    if sr != model_sr:
        print(f'Resampling from {sr} to {MODEL_SR}')
        wav = torchaudio.transforms.Resample(sr, MODEL_SR)(wav)
    return wav


def clean_text(o):
    clean = o.replace(',', ' ')
    clean = clean.strip('\n')
    return clean

def load_transcript(text_file):
    '''Loads a transcript in `text_file`.
    
    Assumes one transcription per line.
    '''
    # load text file
    lines = open(text_file, encoding="utf8").readlines()
    # cleanup the text
    lines = [clean_text(line) for line in lines]
    # replace spaces with special token `|`
    lines = ['|'.join(line.split(' ')) for line in lines]
    # make all characters upper-case for wav2vec token outputs
    lines = [line.upper() for line in lines]
    return lines
    
    
def load_model(model_name):
    '''Loads a pytorch `model_name` from torchaudio.
    '''
    bundle = getattr(torchaudio.pipelines, model_name, None)
    if bundle:
        model = bundle.get_model()
    else:
        raise ValueError(f'Could not find model: {model_name}')
    return model, bundle



def get_emissions(model, audio):
    '''Gets token probabilities from `model` for the speech given in `audio`.
    '''
    emissions, _ = model(audio)
    emissions = torch.log_softmax(emissions, dim=-1)
    return emissions


def get_trellis(emission, tokens, blank_id=0):
    num_frame = emission.size(0)
    num_tokens = len(tokens)

    # Trellis has extra diemsions for both time axis and tokens.
    # The extra dim for tokens represents <SoS> (start-of-sentence)
    # The extra dim for time axis is for simplification of the code.
    trellis = torch.empty((num_frame + 1, num_tokens + 1))
    trellis[0, 0] = 0
    trellis[1:, 0] = torch.cumsum(emission[:, 0], 0)
    trellis[0, -num_tokens:] = -float("inf")
    trellis[-num_tokens:, 0] = float("inf")

    for t in range(num_frame):
        trellis[t + 1, 1:] = torch.maximum(
            # Score for staying at the same token
            trellis[t, 1:] + emission[t, blank_id],
            # Score for changing to the next token
            trellis[t, :-1] + emission[t, tokens],
        )
    return trellis


def backtrack(trellis, emission, tokens, blank_id=0):
    # Note:
    # j and t are indices for trellis, which has extra dimensions
    # for time and tokens at the beginning.
    # When referring to time frame index `T` in trellis,
    # the corresponding index in emission is `T-1`.
    # Similarly, when referring to token index `J` in trellis,
    # the corresponding index in transcript is `J-1`.
    j = trellis.size(1) - 1
    t_start = torch.argmax(trellis[:, j]).item()

    path = []
    for t in range(t_start, 0, -1):
        # 1. Figure out if the current position was stay or change
        # Note (again):
        # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
        # Score for token staying the same from time frame J-1 to T.
        stayed = trellis[t - 1, j] + emission[t - 1, blank_id]
        # Score for token changing from C-1 at T-1 to J at T.
        changed = trellis[t - 1, j - 1] + emission[t - 1, tokens[j - 1]]

        # 2. Store the path with frame-wise probability.
        prob = emission[t - 1, tokens[j - 1] if changed > stayed else 0].exp().item()
        # Return token index and time index in non-trellis coordinate.
        path.append(Point(j - 1, t - 1, prob))

        # 3. Update the token
        if changed > stayed:
            j -= 1
            if j == 0:
                break
    else:
        raise ValueError("Failed to align")
    return path[::-1]


def merge_repeats(path, transcript):
    i1, i2 = 0, 0
    segments = []
    while i1 < len(path):
        while i2 < len(path) and path[i1].token_index == path[i2].token_index:
            i2 += 1
        score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
        segments.append(
            Segment(
                transcript[path[i1].token_index],
                path[i1].time_index,
                path[i2 - 1].time_index + 1,
                score,
            )
        )
        i1 = i2
    return segments


# Merge words
def merge_words(segments, separator="|"):
    words = []
    i1, i2 = 0, 0
    while i1 < len(segments):
        if i2 >= len(segments) or segments[i2].label == separator:
            if i1 != i2:
                segs = segments[i1:i2]
                word = "".join([seg.label for seg in segs])
                score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                words.append(Segment(word, segments[i1].start, segments[i2 - 1].end, score))
            i1 = i2 + 1
            i2 = i1
        else:
            i2 += 1
    return words

In [228]:
# example run through


def run_forced_alignment(
    model_name,
    speech_file,
    text_file,
    device=None,
    
):
    
    # set the hardware device
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # load the model
    model, bundle = load_model(model_name)
    model.eval()
    model.to(device)

    # get model labels
    labels = bundle.get_labels()
    dictionary = {c: i for i, c in enumerate(labels)}
    print(f'Aligning wav2vec2 tokens:')
    keys = [f'{k}: {v}' for k,v in dictionary.items()]
    print('\n'.join(keys))

    # load the audio and transcript
    audio = load_audio(speech_file)
    audio = audio.to(device)
    print(f'Audio shape: {audio.shape}')
    
    # read in the transcript
    if os.path.isfile(text_file):
        transcript = load_transcript(text_file)[0]
    else:
        print(f'Note: using "{text_file}" as the transcript')
        transcript = text_file

    # get the token probabilities
    emissions = get_emissions(model, audio)
    emissions = emissions[0].detach().cpu()

    # turn the transcript into tokens
    tokens = [dictionary[c] for c in transcript]

    # populate the trellis
    trellis = get_trellis(emissions, tokens)

    # walkback to find the most likely trellis path
    path = backtrack(trellis, emissions, tokens)

    # merge the paths with repeated labels
    segments = merge_repeats(path, transcript)

    # merge the words based on the separator token '|'
    word_segments = merge_words(segments)
    
    # cleanup the model
    del model
    model = None
    torch.cuda.empty_cache()
    gc.collect()
    
    return {
        'character_segs': segments,
        'word_segs': word_segments,
    }

### Cleaning up some GPU memory

In [229]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [230]:
del model
model = None
torch.cuda.empty_cache()
gc.collect() 

del model_a
model_a = None
torch.cuda.empty_cache()
gc.collect() 

0

In [231]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


### Calling the Force Alignment on the `kitten_full.wav` file

In [232]:
text_file = ts
model_name = MODEL_NAME


segs = run_forced_alignment(
    model_name,
    audio_file_name,
    text_file,
    device='cpu',
)

Aligning wav2vec2 tokens:
-: 0
|: 1
E: 2
T: 3
A: 4
O: 5
N: 6
I: 7
H: 8
S: 9
R: 10
D: 11
L: 12
U: 13
M: 14
W: 15
C: 16
F: 17
G: 18
Y: 19
P: 20
B: 21
V: 22
K: 23
': 24
X: 25
J: 26
Q: 27
Z: 28
Resampling from 24000 to 16000
Audio shape: torch.Size([1, 116566])
Note: using "GOOD|EVENING|EVERYONE|OBAMA|HERE|BRINGING|YOU|THE|LATEST|NEWS|FROM|THE|UPSIDE-DOWN|WORLD" as the transcript


In [233]:
# character segmentation map
segs['character_segs']

[G	(0.90): [52.000000, 54.000000),
 O	(1.00): [54.000000, 57.000000),
 O	(1.00): [57.000000, 58.000000),
 D	(0.83): [58.000000, 60.000000),
 |	(0.57): [60.000000, 64.000000),
 E	(1.00): [64.000000, 65.000000),
 V	(0.50): [65.000000, 67.000000),
 E	(0.51): [67.000000, 69.000000),
 N	(0.70): [69.000000, 73.000000),
 I	(1.00): [73.000000, 74.000000),
 N	(0.52): [74.000000, 76.000000),
 G	(0.62): [76.000000, 78.000000),
 |	(0.77): [78.000000, 82.000000),
 E	(1.00): [82.000000, 83.000000),
 V	(1.00): [83.000000, 85.000000),
 E	(1.00): [85.000000, 86.000000),
 R	(0.70): [86.000000, 88.000000),
 Y	(0.93): [88.000000, 96.000000),
 O	(1.00): [96.000000, 97.000000),
 N	(0.99): [97.000000, 98.000000),
 E	(0.37): [98.000000, 101.000000),
 |	(0.98): [101.000000, 146.000000),
 O	(0.91): [146.000000, 150.000000),
 B	(1.00): [150.000000, 155.000000),
 A	(0.99): [155.000000, 159.000000),
 M	(0.99): [159.000000, 162.000000),
 A	(0.75): [162.000000, 166.000000),
 |	(0.94): [166.000000, 167.000000),
 H	(1

In [234]:
# view a few chunked words
chunked_words[4]

[['H', ['HH']], ['E', ['IY']], ['RE', ['R']]]

# HERE: MAPPING CHARACTER DURATIONS TO PHONES

In [235]:
char_segs = segs['character_segs'][:]
char_ptr = 0


def frame_to_ms(frame):
    return int(frame * 2) # * 20

# stores the info for the needed transcriptions
full_info = []

# step through the chunked, phoneme words
for cword in chunked_words:
    
    # cleanup punctuation from the word
    word = ''.join(o[0] for o in cword).lower()
    word = word.replace(',', '')
    word = word.replace('.', '')
        
    # alignments for this word
    alns = []
    
    # step through the pairs of letters/phones for this word
    for (letters, phones) in cword:
        
        # cleanup punctuation from the letters
        if letters:
            letters = letters.replace(',', '')
            letters = letters.replace('.', '')
            
        # group the character segments for this letter/phone pair
        cur_segs = []
        while ''.join(o.label for o in cur_segs) != letters and char_ptr < len(char_segs):
            cur_segs.append(char_segs[char_ptr])
            char_ptr += 1
            
        # TODO: handle cases when:
        ## A) There is no phone for the letters
        ## B) There are more than on phones for the letter
        num_phones = len(phones)
        if phones:
            
            # start and stop time of the entire segments
            if cur_segs:
                beg_ms = cur_beg = frame_to_ms(cur_segs[0].start)
                end_ms = frame_to_ms(cur_segs[-1].end)

                dur = end_ms - beg_ms
                dur_per_phone = dur / num_phones

                for idx,phone in enumerate(phones):
                    cur_end = round(cur_beg + dur_per_phone)
                    if cur_segs:
                        alns.append({
                            'ph': phone,
                            'bg': cur_beg,
                            'ed': cur_end,
                        })
                    cur_beg = cur_end
                
        # if no phones are given, then extend the previous phone's durations   
        else:
            if alns and cur_segs:
                prev_seg = alns[-1]
                prev_seg['ed'] = frame_to_ms(cur_segs[-1].end)
                alns[-1] = prev_seg
                
        
    if len(full_info) == 0:
        full_info.append({
            'word': 'sil',
            'phones': [{
                'ph': 'SIL',
                'bg': 0,
                'ed': alns[0]["bg"],
            }]
        })

    # add the info we need for the video generation
    info = {'word': word, 'phones': alns}
    full_info.append(info)
            
    # check if we still have segments to process
    if char_ptr < len(char_segs):
        
        # if we're at the end of a word, add a silence
        if char_segs[char_ptr].label == '|':
            char_ptr += 1
            full_info.append({
                'word': 'sil',
                'phones': [{
                    'ph': 'SIL',
                    'bg': full_info[-1]['phones'][-1]['ed'],
                    'ed': frame_to_ms(char_segs[char_ptr].start),
                }]
            })


In [236]:
full_info

[{'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 0, 'ed': 104}]},
 {'word': 'good',
  'phones': [{'ph': 'G', 'bg': 104, 'ed': 108},
   {'ph': 'UH', 'bg': 108, 'ed': 116},
   {'ph': 'D', 'bg': 116, 'ed': 120}]},
 {'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 120, 'ed': 128}]},
 {'word': 'evening',
  'phones': [{'ph': 'IY', 'bg': 128, 'ed': 130},
   {'ph': 'V', 'bg': 130, 'ed': 138},
   {'ph': 'N', 'bg': 138, 'ed': 146},
   {'ph': 'IH', 'bg': 146, 'ed': 148},
   {'ph': 'NG', 'bg': 148, 'ed': 156}]},
 {'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 156, 'ed': 164}]},
 {'word': 'everyone',
  'phones': [{'ph': 'EH', 'bg': 164, 'ed': 166},
   {'ph': 'V', 'bg': 166, 'ed': 170},
   {'ph': 'IY', 'bg': 170, 'ed': 192},
   {'ph': 'W', 'bg': 192, 'ed': 193},
   {'ph': 'AH', 'bg': 193, 'ed': 194},
   {'ph': 'N', 'bg': 194, 'ed': 202}]},
 {'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 202, 'ed': 292}]},
 {'word': 'obama',
  'phones': [{'ph': 'OW', 'bg': 292, 'ed': 300},
   {'ph': 'B', 'bg': 300, 'ed'

In [237]:
import json

file_name = f"../AAAI22-one-shot-talking-face/samples/phonemes/{base_audio_file_name}.json"
print("Writing to ", file_name)
with open(file_name, "w") as f1:
    json.dump(full_info, f1)
base_audio_file_name
os.getcwd()

Writing to  ../AAAI22-one-shot-talking-face/samples/phonemes/obama_intro.json


'/Users/jfuentes/Projects/infinitenews/ForcedAlignment'

### Group into needed format

In [91]:
import pandas as pd

In [92]:
segs['character_segs'][-3]

O	(1.00): [3944.000000, 3945.000000)

In [93]:
phone_align

NameError: name 'phone_align' is not defined

In [ ]:
s

In [ ]:
[(o.start / 50, o.end / 50) for o in segs['word_segs']]

# Comparing the alignments

In [94]:
result_aligned["word_segments"];

### NODE: older code below

In [101]:
# {'text': 'Good', 'start': 0.10120481927710842, 'end': 0.32385542168674697},
# INTO
# [
#     {
#         "word": "sil",
#         "phones": [
#             {
#                 "ph": "SIL",
#                 "bg": 0,
#                 "ed": 126
#             }
#         ]
#     },
#     {
#         "word": "YOU",
#         "phones": [
#             {
#                 "ph": "Y",
#                 "bg": 126,
#                 "ed": 138
#             },
#             {
#                 "ph": "UW",
#                 "bg": 138,
#                 "ed": 141
#             }
#         ]
#     },
from math import trunc

def makeSegment(word, phones, start, end):
    total = end - start
    time_per_phone = trunc(total / len(phones))
#     print(total, time_per_phone)
    expanded_phones = [{"ph": phone, "bg": start + (i * time_per_phone), "ed": start + ((i + 1) * time_per_phone)} for i, phone in enumerate(phones)]
    return {"word": word, "phones": expanded_phones}

def makeSil(start, end):
    return {"word": "sil", "phones": [{"ph": "SIL", "bg": start, "ed": end}]}
  
all_phonemes = []
for segment in result_aligned["word_segments"]:
    word = segment["text"]
    #Get rid of numbers in phones
    phonemes = wordbreak(word)[0]
    #print(phonemes)
    start_time = trunc(segment["start"] * 100)
    end_time = trunc(segment["end"] * 100)
    
    if len(all_phonemes) > 0:
        last = all_phonemes[-1]
        last_end = last["phones"][-1]["ed"]
        if start_time - last_end > 5:
            sil = makeSil(last_end, start_time)
            all_phonemes.append(sil)
        else:
            start_time = last_end
    else:
        sil = makeSil(0, start_time)
        all_phonemes.append(sil)
        
    segment = makeSegment(word, phonemes, start_time, end_time)
    all_phonemes.append(segment)

all_phonemes[:5]

[{'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 0, 'ed': 10}]},
 {'word': 'Good',
  'phones': [{'ph': 'G', 'bg': 10, 'ed': 14},
   {'ph': 'UH', 'bg': 14, 'ed': 18},
   {'ph': 'D', 'bg': 18, 'ed': 22}]},
 {'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 22, 'ed': 30}]},
 {'word': 'evening,',
  'phones': [{'ph': 'IY', 'bg': 30, 'ed': 34},
   {'ph': 'V', 'bg': 34, 'ed': 38},
   {'ph': 'N', 'bg': 38, 'ed': 42},
   {'ph': 'IH', 'bg': 42, 'ed': 46},
   {'ph': 'NG', 'bg': 46, 'ed': 50}]},
 {'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 50, 'ed': 60}]}]

In [97]:
import json

file_name = "../AAAI22-one-shot-talking-face/samples/phonemes/{name}_og.json".format(name=base_audio_file_name)
print("Writing to ", file_name)
f1 = open(file_name, 'w')
json.dump(all_phonemes, f1)

Writing to  ../AAAI22-one-shot-talking-face/samples/phonemes/kitten_full_og.json


In [ ]:
# all_phonemes